In [30]:
# !pip install keras_cv_attention_models

In [31]:
# import matplotlib.pyplot as plt
# import ghostevals
from tensorflow import keras
import tensorflow as tf
# import IJB_evals
# import keras_cv_attention_models
import GhostFaceNets_with_Bias

In [32]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print(tf.config.list_physical_devices('GPU'))

[]


2 ways to load the model

Stride 1

In [33]:
#Either
model = keras.models.load_model('..\\src\\models\\ghostFaceNet\\GhostFaceNet_W1.3_S1_ArcFace.h5', compile=False)

In [34]:
base_dir = "..\\src\\models\\ghostFaceNet"
tflite_path = base_dir + "\\tflites"
basic_model_dir = tflite_path + "\\basic_model.tflite"
quant_model_dir = tflite_path + "\\quant_model.tflite"


In [35]:
model.summary()

Model: "GhostNet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 112, 112, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 112, 112, 20)         540       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 112, 112, 20)         80        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (PReLU)          (None, 112, 112, 20)         20        ['batch_normalization[0

##### basic convert model

In [36]:
# Convert the model. 
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()


INFO:tensorflow:Assets written to: C:\Users\siedt\AppData\Local\Temp\tmpc12c6mi8\assets


INFO:tensorflow:Assets written to: C:\Users\siedt\AppData\Local\Temp\tmpc12c6mi8\assets


In [37]:

# Save the model.
print('Saving basic model...')
with open(basic_model_dir, 'wb') as f:
  f.write(tflite_model)

Saving basic model...


##### default optimization (quantization)

In [44]:
del converter
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter._experimental_lower_tensor_list_ops = False
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()


INFO:tensorflow:Assets written to: C:\Users\siedt\AppData\Local\Temp\tmp7kbojzad\assets


INFO:tensorflow:Assets written to: C:\Users\siedt\AppData\Local\Temp\tmp7kbojzad\assets


In [46]:

# Save the model.
print('Saving quanized model...')
with open(quant_model_dir, 'wb') as f:
  f.write(tflite_model)

Saving quanized model...
